In [1]:
!nvidia-smi
import torch
torch.cuda.empty_cache()

Sun Aug 29 18:43:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.73.01    Driver Version: 460.73.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
from os.path import exists, join, basename, splitext

git_repo = 'https://github.com/sfoucher/ConvPoint'
#git_repo = 'https://github.com/mpelchat04/ConvPoint'

if not os.path.isdir("ConvPoint"):
  !git clone {git_repo}
else:
  !git pull

fatal: not a git repository (or any parent up to mount point /)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


In [ ]:
%%capture
%%bash
cd /content/ConvPoint/convpoint/knn/
#%pycat setup.py
apt-get install gcc g++ -y -qq
python setup.py install --home="."

In [19]:
%%capture --no-stderr
%cd /home/jovyan/ConvPoint
!pip install -e .
!pip install laspy==1.7.*

In [16]:
import sys
sys.path.append('/home/jovyan/ConvPoint')
import convpoint

In [5]:
%%capture
%%bash
cd /home/jovyan
rm -rf LASzip_src
git clone https://github.com/LASzip/LASzip.git LASzip_src
cd LASzip_src
git checkout 3.1.0
cmake .
make
make install
cd ..

In [7]:
%%bash
echo "downloading:  ..." && curl --progress-bar --output /home/jovyan/LAStools.zip --location https://www.cs.unc.edu/~isenburg/lastools/download/LAStools.zip
ls -l

downloading:  ...
total 56
drwxr-sr-x 5 jovyan users 4096 Aug 29 18:06 convpoint
drwxr-sr-x 2 jovyan users 4096 Aug 29 18:06 ConvPoint.egg-info
-rw-r--r-- 1 jovyan users 2721 Aug 29 17:56 convpoint-env2.yml
-rw-r--r-- 1 jovyan users 1178 Aug 29 17:56 convpoint-env.yml
drwxr-sr-x 2 jovyan users 4096 Aug 29 17:56 doc
-rw-r--r-- 1 jovyan users 2520 Aug 29 17:56 env.yml
drwxr-sr-x 8 jovyan users 4096 Aug 29 17:56 examples
-rw-r--r-- 1 jovyan users  216 Aug 29 17:56 global_tags.py
-rw-r--r-- 1 jovyan users 1516 Aug 29 17:56 LICENSE.md
drwxr-sr-x 2 jovyan users 4096 Aug 29 17:56 models
drwxr-sr-x 2 jovyan users 4096 Aug 29 17:56 networks
-rw-r--r-- 1 jovyan users 2515 Aug 29 17:56 README.md
-rw-r--r-- 1 jovyan users  335 Aug 29 17:56 setup.py
drwxr-sr-x 2 jovyan users 4096 Aug 29 17:56 utils


######################################################################## 100.0%


In [9]:
#@markdown Step 5: installation of LAStools
%%capture
%%bash
cd /home/jovyan
unzip LAStools.zip
cd LAStools
make
cp bin/laszip /usr/local/bin
cp /home/jovyan/LAStools/bin/las2las /usr/local/bin
cp /home/jovyan/LAStools/bin/lasinfo /usr/local/bin
ln -s /usr/local/bin/laszip /usr/local/bin/laszip-cli

SyntaxError: invalid syntax (778758779.py, line 5)

In [11]:
!mkdir -p /home/jovyan/datasets/trn /home/jovyan/datasets/val /home/jovyan/datasets/tst /home/jovyan/datasets/results

In [2]:
%run /home/jovyan/ConvPoint/examples/airborne_lidar/prepare_airborne_lidar_label --folder /home/jovyan/DALES/dales_las --dest /home/jovyan/datasets

/home/jovyan/DALES/dales_las/train/5080_54435.las


/home/jovyan/ConvPoint/examples/airborne_lidar/prepare_airborne_lidar_label.py:55: RuntimeWarning: invalid value encountered in true_divide
  norm_intensity = (intensity - np.min(intensity)) / (np.max(intensity) - np.min(intensity))


File train/5080_54435.las prepared. 11,345,691 points written.
/home/jovyan/DALES/dales_las/train/5085_54320.las
File train/5085_54320.las prepared. 14,394,749 points written.
/home/jovyan/DALES/dales_las/train/5095_54440.las
File train/5095_54440.las prepared. 11,703,345 points written.
/home/jovyan/DALES/dales_las/train/5095_54455.las
File train/5095_54455.las prepared. 13,561,343 points written.
/home/jovyan/DALES/dales_las/train/5100_54495.las
File train/5100_54495.las prepared. 12,085,953 points written.
/home/jovyan/DALES/dales_las/train/5105_54405.las
File train/5105_54405.las prepared. 11,915,905 points written.
/home/jovyan/DALES/dales_las/train/5105_54460.las
File train/5105_54460.las prepared. 12,954,374 points written.
/home/jovyan/DALES/dales_las/train/5110_54320.las
File train/5110_54320.las prepared. 17,747,769 points written.
/home/jovyan/DALES/dales_las/train/5110_54460.las
File train/5110_54460.las prepared. 13,784,200 points written.
/home/jovyan/DALES/dales_las/trai

In [12]:
!grid run /home/jovyan/ConvPoint/examples/airborne_lidar/airborne_lidar_seg.py --savedir /home/jovyan/datasets/results --rootdir /home/jovyan/datasets --batchsize 16 --num_workers 4

GraphQL URL: https://platform.grid.ai:443/graphql
Traceback (most recent call last):-467 ...
  File "/home/jovyan/conda/bin/grid", line 8, in <module>
    sys.exit(main())
  File "/home/jovyan/conda/lib/python3.8/site-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/home/jovyan/conda/lib/python3.8/site-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/home/jovyan/conda/lib/python3.8/site-packages/click/core.py", line 1259, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/home/jovyan/conda/lib/python3.8/site-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/home/jovyan/conda/lib/python3.8/site-packages/click/core.py", line 610, in invoke
    return callback(*args, **kwargs)
  File "/home/jovyan/conda/lib/python3.8/site-packages/click/decorators.py", line 21, in new_func
    return f(get_current_context(), *args, **kwargs)
  File "

In [13]:
%run /home/jovyan/ConvPoint/examples/airborne_lidar/airborne_lidar_seg.py --savedir /home/jovyan/datasets/results --rootdir /home/jovyan/datasets --batchsize 16 --num_workers 4

Create file list...
Las files per dataset:
 Trn: 26 
 Val: 3 
 Tst: 11
Creating network...
Model with dropout
Number of parameters in the model: 2,329,657
Creating dataloader and optimizer...done
Creating results folder...done at /home/jovyan/datasets/results/SegBig_8168_drop0_2021-08-29-19-11-42


Epoch 0:  40%|████████████████████▍                              | 240/600 [19:54<29:52,  4.98s/it, AA=0.2943, IOU=0.2660, LOSS=3.3695e-06, OA=0.8897]


KeyboardInterrupt: 

In [18]:
!python /home/jovyan/ConvPoint/run.py --savedir /home/jovyan/datasets/results --rootdir /home/jovyan/datasets --batchsize 16 --num_workers 4

Traceback (most recent call last):
  File "/home/jovyan/ConvPoint/run.py", line 11, in <module>
    from convpoint.examples.airborne_lidar.airborne_lidar_utils import write_features
ModuleNotFoundError: No module named 'convpoint.examples'


In [33]:
las = laspy.read('/home/jovyan/DALES/dales_las/train/5080_54435.las')
las.header

<LasHeader(1.2, <PointFormat(1, 0 bytes of extra dims)>)>

In [ ]:
las.header.